# DESCRIPTION
In this project, I scraped the data available of a user `nickname808` which has information about the offers , transactions of NFTs that the user performed.\
Data contains information about the name, collection, price in digital currency, price in Dollar , time of activity , etc.\
\
\
\
**NOTE** 
Some columns have `Not Available` values because the information isnt available on the website. Not to be confused with null values.

In [25]:
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import numpy as np
import time
from datetime import timedelta, datetime
import itertools

pd.set_option('display.max_rows', None)

In [26]:
url = "https://opensea.io/nickname808/activity?search[eventTypes][0]=OFFER_ENTERED&search[eventTypes][1]=AUCTION_CREATED"

In [27]:
driver = webdriver.Chrome()   #generate a driver
driver.get(url)               #going to the url
driver.maximize_window()     #maximizes the window
time.sleep(2)                #waits 2 seconds after loading
screen_height = driver.execute_script("return window.screen.height;") #get current screen height
i = 1
sol = ""
#This loop will run until we reach the end of the infinite scroll
while True:
    #scroll the page to the next screen height one by one
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
    i += 1
    #get all the text values
    x = driver.find_elements('xpath', '//*[@id="main"]/div/div/div/div[5]/div/div[5]/div[3]/div[2]/div/div[2]')
    sol = sol + x[0].text
    sol = sol + "\n"
    time.sleep(3)
    #get current scrollHeight
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    #break the loop if the screen height exceeds the scroll height
    if (screen_height) * i > scroll_height:
        break

In [28]:
driver.close() #closing the driver

In [29]:
lst_off = ["List", "Offer"]

In [30]:
col = sol.split("\n") #splitting the string to list

In [31]:
indices = [i for i, x in enumerate(col) if x in lst_off] #get indices of "Offer" or "List" as they serve as a starting point

In [32]:
print(indices)

[0, 10, 21, 32, 43, 54, 65, 76, 87, 98, 109, 120, 130, 141, 152, 163, 174, 185, 196, 207, 218, 229, 240, 252, 264, 276, 288, 300, 312, 324, 336, 348, 360, 372, 384, 396, 408, 420, 431, 443, 455, 467, 479, 491, 503, 515, 527, 539, 551, 563, 575, 587, 599, 611, 623, 635, 647, 659, 671, 683, 695, 707, 719, 731, 743, 755, 767, 779, 791, 803, 815, 827, 839, 851, 863, 875, 887, 899, 911, 923, 935, 947, 959, 971, 983, 995, 1007, 1019, 1031, 1043, 1055, 1067, 1079, 1091, 1103, 1115, 1127, 1139, 1151, 1163, 1175, 1187, 1199, 1211, 1223, 1235, 1247, 1259, 1271, 1283, 1295, 1307, 1319, 1331, 1343, 1355, 1367, 1379, 1391, 1403, 1415, 1427, 1439, 1451, 1463, 1475, 1487, 1499, 1511, 1523, 1535, 1547, 1559, 1571, 1583, 1595, 1607, 1619, 1631, 1643, 1655, 1667, 1679, 1690, 1701, 1713, 1725, 1737, 1749, 1761, 1772, 1783, 1795, 1806, 1818, 1828, 1840, 1851, 1862, 1874, 1886, 1897, 1908, 1920, 1932, 1944, 1955, 1966, 1977, 1989, 2001, 2011, 2023, 2034, 2045, 2056, 2067, 2078, 2090, 2101, 2112, 2124, 2135

In [33]:
new_col = []

In [34]:
# append all the lists to a new list
for i in range(len(indices)-1):
    new_col.append(col[indices[i] : indices[i+1]])
new_col.append(col[indices[i+1] : len(col)])

In [35]:
len(new_col)

187

In [36]:
new_col

[['Offer',
  'Lunar Calendar',
  'Sunflower Land Collectibles',
  '4.600 USDC',
  '$4.61',
  '--',
  '10',
  'nickname808',
  '---',
  '3d ago'],
 ['Offer',
  'Accepted',
  'Lunar Calendar',
  'Sunflower Land Collectibles',
  '5 USDC',
  '$5.01',
  '--',
  '10',
  'nickname808',
  '---',
  '3d ago'],
 ['Offer',
  'Canceled',
  'Tiki Totem',
  'Sunflower Land Collectibles',
  '4.350 USDC',
  '$4.36',
  '--',
  '10',
  'nickname808',
  '---',
  '3d ago'],
 ['Offer',
  'Canceled',
  'Lunar Calendar',
  'Sunflower Land Collectibles',
  '4.500 USDC',
  '$4.51',
  '--',
  '10',
  'nickname808',
  '---',
  '3d ago'],
 ['Offer',
  'Canceled',
  'Iron Idol',
  'Sunflower Land Collectibles',
  '580 USDC',
  '$581.74',
  '--',
  '1',
  'nickname808',
  '---',
  '3d ago'],
 ['Offer',
  'Canceled',
  'Carrot Sword',
  'Sunflower Land Collectibles',
  '115 USDC',
  '$115.34',
  '--',
  '2',
  'nickname808',
  '---',
  '3d ago'],
 ['Offer',
  'Canceled',
  'Tiki Totem',
  'Sunflower Land Collectibles

In [37]:
status = ['Accepted', 'Canceled', 'Expired', 'Sold']

In [38]:
# add "Not Available" to the lists whose status is not given
for n in new_col:
    if n[1] not in status:
        n.insert(1, 'Not Available')

In [39]:
#replacing the values that are not avilable with "Not Available"
for n in new_col:
    n[6] = "Not Available"
    n[9] = "Not Available"

In [40]:
# removing last values "sell" and "" to make all the lists same length
for n in new_col:
    err = ['sell', '']
    if n[-1] in err:
        n.remove(n[-1])

In [41]:
nyc = []

In [42]:
for n_c in new_col:
    if n_c not in nyc:
        nyc.append(n_c)
    else:
        continue

In [43]:
len(nyc)

133

In [98]:
#2d list to a DF
df = pd.DataFrame(nyc, columns = ['Event Type', 'Status', 'NFT', 'Collection', 'Price', 'Price (in US dollar)', 'Rarity', 'Quantity', 'From', 'To', 'Time'])

In [99]:
df.head()

,Event Type,Status,NFT,Collection,Price,Price (in US dollar),Rarity,Quantity,From,To,Time
0,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,4.600 USDC,$4.61,Not Available,10,nickname808,Not Available,3d ago
1,Offer,Accepted,Lunar Calendar,Sunflower Land Collectibles,5 USDC,$5.01,Not Available,10,nickname808,Not Available,3d ago
2,Offer,Canceled,Tiki Totem,Sunflower Land Collectibles,4.350 USDC,$4.36,Not Available,10,nickname808,Not Available,3d ago
3,Offer,Canceled,Lunar Calendar,Sunflower Land Collectibles,4.500 USDC,$4.51,Not Available,10,nickname808,Not Available,3d ago
4,Offer,Canceled,Iron Idol,Sunflower Land Collectibles,580 USDC,$581.74,Not Available,1,nickname808,Not Available,3d ago


In [71]:
df.shape

(133, 11)

# Getting Timestamp

Running the driver again to get timestamps but this time the approach is different from the previous one\
To ensure that we get the timestamp for the correct nft we will use two helper features , `price` and `time`

Timestamp of an activity is available only after hovering over the element in which it is contained.\
Now, the webpage loaded the container elements (that contained the element to be hovered over) at random positions within their parent element\
which created an issue in extracting the texts leading to the creation of few null values. But thankfully, the replacement of null values can be found with\
the help of `Time` column and were pretty much close to the original value.

In [72]:
driver = webdriver.Chrome() #generate a driver
driver.get(url)            
action = ActionChains(driver) #going to the url
driver.maximize_window()     #maximizes the window
time.sleep(2)                #waits 2 seconds after loading
screen_height = driver.execute_script("return window.screen.height;") #get current screen height
i = 1
prices = []
when = []
tyms = []
#This loop will run until we reach the end of the infinite scroll
while True:
    #scroll the page to the next screen height one by one
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
    i += 1
    #get all the text values
    time.sleep(3)
    
    elems = []
    divs = []
    
    for div in driver.find_elements(By.TAG_NAME, "div"):
        if div.get_attribute("role") == "listitem":
            divs.append(div)

    for div in divs:
        try:
            for elem in div.find_elements(By.TAG_NAME, "div"):
                if elem.get_attribute("class") == "sc-fe5f9c83-0 mGAUR Price--fiat-amount":
                    prices.append(elem.text)
        except:
            prices.append("null")
            tyms.append("null")
            when.append("null")
            continue
            
        for elem in div.find_elements(By.TAG_NAME, "span"):
            if elem.get_attribute("aria-expanded") == "false":
                when.append(elem.text)
                action.move_to_element(elem).perform()
                try:
                    tyms.append(driver.find_element(By.CLASS_NAME, "tippy-content").text)
                except:
                    tyms.append("null")
            
            
    #get current scrollHeight
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    #break the loop if the screen height exceeds the scroll height
    if (screen_height) * i > scroll_height:
        break

In [73]:
driver.close()

In [74]:
len(tyms), len(when), len(prices)

(227, 227, 227)

In [75]:
check = []

In [76]:
# concat the three features to check for the duplicate entries
for t,w,p in zip(tyms, when, prices):
    s = t +"  -  "+w+"  -  "+p
    if s not in check:
        check.append(s)
    else:
        continue

In [77]:
len(check)

83

In [78]:
# remove the values that have null timestamp in it
for c in check:
    if c[:4] == "null":
        check.remove(c)

In [79]:
check[:2]

['March 20, 2023 at 10:58 PM  -  3d ago  -  $4.61',
 'March 20, 2023 at 5:43 PM  -  3d ago  -  $5.01']

In [100]:
# use the previously concatenated string to fill the values of timestamps accordingly
for c in check:
    ls = c.split("  -  ")
    df.loc[(df["Price (in US dollar)"] == ls[2]) & (df["Time"] == ls[1]), "Timestamp"] = ls[0]   

In [101]:
df.head()

,Event Type,Status,NFT,Collection,Price,Price (in US dollar),Rarity,Quantity,From,To,Time,Timestamp
0,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,4.600 USDC,$4.61,Not Available,10,nickname808,Not Available,3d ago,"March 20, 2023 at 7:47 PM"
1,Offer,Accepted,Lunar Calendar,Sunflower Land Collectibles,5 USDC,$5.01,Not Available,10,nickname808,Not Available,3d ago,"March 20, 2023 at 5:43 PM"
2,Offer,Canceled,Tiki Totem,Sunflower Land Collectibles,4.350 USDC,$4.36,Not Available,10,nickname808,Not Available,3d ago,"March 20, 2023 at 5:15 PM"
3,Offer,Canceled,Lunar Calendar,Sunflower Land Collectibles,4.500 USDC,$4.51,Not Available,10,nickname808,Not Available,3d ago,"March 20, 2023 at 6:16 PM"
4,Offer,Canceled,Iron Idol,Sunflower Land Collectibles,580 USDC,$581.74,Not Available,1,nickname808,Not Available,3d ago,"March 20, 2023 at 1:41 PM"


In [102]:
# check for the null values or the rows whose timestamp hasn't been found
df[df['Timestamp'].isna()].head()

,Event Type,Status,NFT,Collection,Price,Price (in US dollar),Rarity,Quantity,From,To,Time,Timestamp
12,Offer,Canceled,Iron Idol,Sunflower Land Collectibles,453 USDC,$454.36,Not Available,1,nickname808,Not Available,6d ago,NaN
13,Offer,Canceled,Iron Idol,Sunflower Land Collectibles,460 USDC,$461.38,Not Available,1,nickname808,Not Available,4d ago,NaN
14,Offer,Canceled,Iron Idol,Sunflower Land Collectibles,458 USDC,$459.37,Not Available,1,nickname808,Not Available,4d ago,NaN
15,Offer,Canceled,Iron Idol,Sunflower Land Collectibles,480 USDC,$481.44,Not Available,1,nickname808,Not Available,3d ago,NaN
21,List,Sold,Kale,Sunflower Land Collectibles,0.044 MATIC,$0.05,Not Available,77,nickname808,Not Available,2mo ago,NaN


To fill the null values , i used the `Time` column whose entries looked like `3d ago` and got the estimate timestamp for each missing values

In [103]:
t = list(df.loc[df['Timestamp'].isna()]["Time"].values)

In [113]:
stamps = []

In [114]:
# convert the datetime to required format and filling the null values
hr = 1
mn = 2
i = 1
for tym in t:
    now = datetime.now()
    if tym[-5] == 'o':
        m = int(''.join(x for x in tym if x.isdigit()))
        tm = now - timedelta(weeks=m*4)
        s = tm.strftime("%m %d, %Y at %I:%M %p")
        if s not in stamps:
            stamps.append(s)
        else:
            tm = tm - timedelta(hours = hr, minutes = mn)
            stamps.append(tm.strftime("%m %d, %Y at %I:%M %p"))
            hr += 1
            mn += 3
        i +=1
        
    elif tym[-5] == 'd':
        d = int(''.join(x for x in tym if x.isdigit()))
        tm = now - timedelta(days = d)
        s = tm.strftime("%m %d, %Y at %I:%M %p")
        if s not in stamps:
            stamps.append(s)
        else:
            tm = tm - timedelta(hours = hr, minutes = mn)
            stamps.append(tm.strftime("%m %d, %Y at %I:%M %p"))
            hr += 1
            mn += 3
        i +=1
        
    elif tym[-5] == 'h':
        h = int(''.join(x for x in tym if x.isdigit()))
        tm = now - timedelta(hours = h)
        s = tm.strftime("%m %d, %Y at %I:%M %p")
        if s not in stamps:
            stamps.append(s)
        else:
            tm = tm - timedelta(hours = hr, minutes = mn)
            stamps.append(tm.strftime("%m %d, %Y at %I:%M %p"))
            hr += 1
            mn += 3
        i +=1

In [115]:
month = {'01':'January',
		'02':'February',
		'03':'March',
		'04':'April',
		'05':'May',
		'06':'June',
		'07':'July',
		'08':'August',
		'09':'September',
		'10':'October',
		'11':'November',
		'12':'December'	}

In [116]:
up_stamps = []

In [117]:
#converting month number to name
for tym in stamps:
    mnth = month[tym[0:2]]
    up_stamps.append(mnth + tym[2:])

In [118]:
# replacing null values with new values
df.loc[df['Timestamp'].isna(), "Timestamp"] = up_stamps

In [120]:
df.isnull().sum()

Event Type              0
Status                  0
NFT                     0
Collection              0
Price                   0
Price (in US dollar)    0
Rarity                  0
Quantity                0
From                    0
To                      0
Time                    0
Timestamp               0
dtype: int64

In [126]:
df.tail()

,Event Type,Status,NFT,Collection,Price,Price (in US dollar),Rarity,Quantity,From,To,Time,Timestamp
128,List,Not Available,Billionaire Skull Club (ETH) #7684,BillionaireSkullClub,0.087 ETH,$160.51,Not Available,1,nickname808,Not Available,11mo ago,"May 18, 2022 at 12:44 AM"
129,List,Sold,Stone Pickaxe,Sunflower Land Collectibles,0.0017 ETH,$3.04,Not Available,1,nickname808,Not Available,11mo ago,"May 17, 2022 at 11:41 PM"
130,List,Not Available,Wheat,Sunflower Land Collectibles,0 ETH,$0.00,Not Available,2.2E-17,nickname808,Not Available,11mo ago,"April 25, 2022 at 4:40 AM"
131,List,Not Available,Noox Genesis Pass: Tier 3 299/300,Noox Genesis Pass Tier 3,0.1481 ETH,$273.21,Not Available,1,nickname808,Not Available,11mo ago,"May 17, 2022 at 10:38 PM"
132,List,Not Available,Billionaire Skull Club (ETH) #7684,BillionaireSkullClub,0.009 ETH,$16.60,Not Available,1,nickname808,Not Available,11mo ago,"May 17, 2022 at 09:35 PM"


In [123]:
df.to_csv("nickname808.csv")

In [125]:
df.to_excel("nickname808.xls")

<ipython-input-125-18c46dadb685>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel("nickname808.xls")
